In [61]:
import pandas as pd
import numpy as np
import joblib

from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [62]:
# Pas dit aan als je CSV ergens anders staat
CSV_PATH = "data/Avanskeuzecompass.modules.csv"

# Dit is de runtime-locatie waar FastAPI volgens .env naar kijkt:
# MODEL_ARTIFACT_PATH=ml/artifacts/current  -> verwacht recommender.joblib daarbinnen
OUTPUT_PATH = Path("../artifacts/current/recommender.joblib")

In [63]:
df = pd.read_csv(CSV_PATH)
df.head()

,_id,name,shortdescription,description,content,studycredit,location,contact_id,level,learningoutcomes,module_tags,popularity_score,estimated_difficulty,available_spots,start_date
0,695a3402b5b8285c7c75dc71,Kennismaking met Psychologie,"Brein, gedragsbeinvloeding, ontwikkelingspsych...",In deze module leer je hoe je gedrag van jezel...,In deze module leer je hoe je gedrag van jezel...,15,Den Bosch,58,NLQF5,A. Je beantwoordt vragen in een meerkeuze kenn...,"['brein', 'gedragsbeinvloeding', 'ontwikkeling...",319,1,79,2026-12-24T00:00:00.000Z
1,695a3402b5b8285c7c75dc72,Learning and working abroad,"Internationaal, persoonlijke ontwikkeling, ver...",Studenten kiezen binnen de (stam) van de oplei...,Studenten kiezen binnen de (stam) van de oplei...,15,Den Bosch,58,NLQF5,De student toont professioneel gedrag conform ...,"['internationaal', 'persoonlijke', 'ontwikkeli...",172,5,56,2026-12-20T00:00:00.000Z
2,695a3402b5b8285c7c75dc73,Proactieve zorgplanning,"Proactieve zorgplanning, cocreatie, ziekenhuis",Het Jeroen Bosch ziekenhuis wil graag samen me...,Het Jeroen Bosch ziekenhuis wil graag samen me...,15,Den Bosch,59,NLQF5,De student past pro actieve zorgplanning toe b...,"['proactieve', 'zorgplanning', 'cocreatie', 'z...",217,5,55,2026-09-23T00:00:00.000Z
3,695a3402b5b8285c7c75dc74,Rouw en verlies,"Rouw & verlies, palliatieve zorg & redeneren, ...",In deze module wordt stil gestaan bij rouw en ...,In deze module wordt stil gestaan bij rouw en ...,30,Den Bosch,58,NLQF6,De student regisseert en voert (deels) zelfsta...,"['rouw', 'verlies', 'palliatieve', 'zorg', 're...",454,1,54,2026-10-25T00:00:00.000Z
4,695a3402b5b8285c7c75dc75,Acuut complexe zorg,"Acute zorg, complexiteit, ziekenhuis, revalidatie",In deze module kunnen studenten zich verdiepen...,In deze module kunnen studenten zich verdiepen...,30,Den Bosch,58,NLQF6,De student regisseert en voert (deels) zelfsta...,"['acute', 'zorg', 'complexiteit', 'ziekenhuis'...",178,5,38,2026-11-19T00:00:00.000Z


In [64]:
df.info()
df.columns.tolist()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   _id                   201 non-null    object
 1   name                  201 non-null    object
 2   shortdescription      175 non-null    object
 3   description           199 non-null    object
 4   content               199 non-null    object
 5   studycredit           201 non-null    int64 
 6   location              201 non-null    object
 7   contact_id            201 non-null    int64 
 8   level                 201 non-null    object
 9   learningoutcomes      140 non-null    object
 10  module_tags           201 non-null    object
 11  popularity_score      201 non-null    int64 
 12  estimated_difficulty  201 non-null    int64 
 13  available_spots       201 non-null    int64 
 14  start_date            201 non-null    object
dtypes: int64(5), object(10)
memory usage: 23

['_id',
 'name',
 'shortdescription',
 'description',
 'content',
 'studycredit',
 'location',
 'contact_id',
 'level',
 'learningoutcomes',
 'module_tags',
 'popularity_score',
 'estimated_difficulty',
 'available_spots',
 'start_date']

In [65]:
# Zorg dat _id altijd string is (Mongo ObjectId string)
if "_id" not in df.columns:
    raise ValueError("CSV mist kolom _id. Deze is verplicht voor backend-integratie.")

df["_id"] = df["_id"].astype(str)

# studycredit netjes numeriek (nullable)
if "studycredit" in df.columns:
    df["studycredit"] = pd.to_numeric(df["studycredit"], errors="coerce").astype("Int64")

In [66]:
TEXT_COLUMNS = [
    "name",
    "shortdescription",
    "description",
    "content",
    "learningoutcomes",
    "module_tags",
]

available_text_cols = [c for c in TEXT_COLUMNS if c in df.columns]
if not available_text_cols:
    raise ValueError("Geen tekstkolommen gevonden voor TF-IDF.")

df["text"] = (
    df[available_text_cols]
    .fillna("")
    .astype(str)
    .agg(" ".join, axis=1)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

df[["name", "text"]].head()

,name,text
0,Kennismaking met Psychologie,"Kennismaking met Psychologie Brein, gedragsbei..."
1,Learning and working abroad,"Learning and working abroad Internationaal, pe..."
2,Proactieve zorgplanning,Proactieve zorgplanning Proactieve zorgplannin...
3,Rouw en verlies,"Rouw en verlies Rouw & verlies, palliatieve zo..."
4,Acuut complexe zorg,"Acuut complexe zorg Acute zorg, complexiteit, ..."


In [67]:
before = len(df)
df = df[df["text"].str.len() > 0].reset_index(drop=True)
after = len(df)

print(f"Verwijderd: {before - after} modules zonder tekst")

Verwijderd: 0 modules zonder tekst


In [68]:
tfidf = TfidfVectorizer(
    lowercase=True,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.95,
)

X_tfidf = tfidf.fit_transform(df["text"])
X_tfidf.shape

(201, 3479)

In [69]:
i = 0  # index van een module
query_name = df.loc[i, "name"]

sims = cosine_similarity(X_tfidf[i], X_tfidf).flatten()
top_idx = np.argsort(sims)[::-1][:10]

print("Query:", query_name)
df.loc[top_idx, ["_id", "name"]]

Query: Kennismaking met Psychologie


,_id,name
0,695a3402b5b8285c7c75dc71,Kennismaking met Psychologie
51,695a3402b5b8285c7c75dca4,Act For Change Together NLQF6 30 + 15 ECTS
179,695a3402b5b8285c7c75dd24,The Creative Agency
185,695a3402b5b8285c7c75dd2a,Veranderen is Mensenwerk
177,695a3402b5b8285c7c75dd22,Inclusief samenwerken zonder verliezers
188,695a3402b5b8285c7c75dd2d,Stopmotion
56,695a3402b5b8285c7c75dca9,Branding: Strategisch Merkenmanagement
16,695a3402b5b8285c7c75dc81,Focus op jeugd
62,695a3402b5b8285c7c75dcaf,BI Innovatie
64,695a3402b5b8285c7c75dcb1,BI Onderzoek voor innovatie


In [70]:
ARTIFACT_COLUMNS = [
    "_id",
    "name",
    "location",
    "level",
    "studycredit",
    "text",
]

artifact_df = df[[c for c in ARTIFACT_COLUMNS if c in df.columns]].copy()

# 🔒 Forceer correcte Mongo-koppeling:
# FastAPI code pakt row.get("module_id") eerst, dus we maken die expliciet gelijk aan Mongo _id.
artifact_df["_id"] = artifact_df["_id"].astype(str)
artifact_df["module_id"] = artifact_df["_id"]

# Verwijder eventuele numerieke id-kolom als die bestaat (extra safety)
if "id" in artifact_df.columns:
    artifact_df = artifact_df.drop(columns=["id"])

artifact_df.head()

,_id,name,location,level,studycredit,text,module_id
0,695a3402b5b8285c7c75dc71,Kennismaking met Psychologie,Den Bosch,NLQF5,15,"Kennismaking met Psychologie Brein, gedragsbei...",695a3402b5b8285c7c75dc71
1,695a3402b5b8285c7c75dc72,Learning and working abroad,Den Bosch,NLQF5,15,"Learning and working abroad Internationaal, pe...",695a3402b5b8285c7c75dc72
2,695a3402b5b8285c7c75dc73,Proactieve zorgplanning,Den Bosch,NLQF5,15,Proactieve zorgplanning Proactieve zorgplannin...,695a3402b5b8285c7c75dc73
3,695a3402b5b8285c7c75dc74,Rouw en verlies,Den Bosch,NLQF6,30,"Rouw en verlies Rouw & verlies, palliatieve zo...",695a3402b5b8285c7c75dc74
4,695a3402b5b8285c7c75dc75,Acuut complexe zorg,Den Bosch,NLQF6,30,"Acuut complexe zorg Acute zorg, complexiteit, ...",695a3402b5b8285c7c75dc75


In [71]:
artifact = {
    "tfidf": tfidf,
    "df": artifact_df,
    "X_tfidf": X_tfidf,
}

In [72]:
assert "_id" in artifact_df.columns, "❌ _id ontbreekt"
assert "module_id" in artifact_df.columns, "❌ module_id ontbreekt"
assert "name" in artifact_df.columns, "❌ name ontbreekt"
assert X_tfidf.shape[0] == len(df), "❌ matrix/df mismatch (X_tfidf en df moeten dezelfde lengte hebben)"

# module_id moet objectid-achtig zijn: 24 hex chars (niet verplicht, maar nuttige check)
bad_ids = artifact_df["module_id"].astype(str).str.len().ne(24).sum()
print(f"Let op: {bad_ids} module_id's zijn niet 24 chars (kan ok zijn als jullie IDs anders zijn).")

print("✅ Artifact voldoet aan FastAPI + backend contract")

Let op: 0 module_id's zijn niet 24 chars (kan ok zijn als jullie IDs anders zijn).
✅ Artifact voldoet aan FastAPI + backend contract


In [73]:
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)

joblib.dump(artifact, OUTPUT_PATH)

print("✅ Artifact opgeslagen op:", OUTPUT_PATH.resolve())

✅ Artifact opgeslagen op: C:\Users\serin\Desktop\Studie\LU3\LU3-Minimum-Viable-Product\ai-service\ml\artifacts\current\recommender.joblib


In [74]:
loaded = joblib.load(OUTPUT_PATH)
ldf = loaded["df"]

print("Loaded df columns:", list(ldf.columns))
print("Sample module_id:", ldf["module_id"].head(3).tolist())
print("Sample _id:", ldf["_id"].head(3).tolist())

Loaded df columns: ['_id', 'name', 'location', 'level', 'studycredit', 'text', 'module_id']
Sample module_id: ['695a3402b5b8285c7c75dc71', '695a3402b5b8285c7c75dc72', '695a3402b5b8285c7c75dc73']
Sample _id: ['695a3402b5b8285c7c75dc71', '695a3402b5b8285c7c75dc72', '695a3402b5b8285c7c75dc73']
